In [2]:
import rasterio

# Open the TIFF file
with rasterio.open("super_res_output.tif") as src:
    for i in range(1, src.count + 1):  # Band indices start from 1
        band = src.read(i)
        
        # Save each band to a separate file
        out_meta = src.meta.copy()
        out_meta.update({
            "count": 1  # Only one band
        })

        with rasterio.open(f"band_{i}.tif", "w", **out_meta) as dst:
            dst.write(band, 1)


In [3]:
import rasterio
import numpy as np

# Define input file paths (adjust if filenames differ)
b01_path = 'band_1.tif'  # Coastal band (443 nm)
b03_path = 'band_3.tif'  # Green band (560 nm)

# Load B01 and B03
with rasterio.open(b01_path) as src:
    B01 = src.read(1).astype('float32')
    profile = src.profile  # Get metadata for saving output

with rasterio.open(b03_path) as src:
    B03 = src.read(1).astype('float32')

# Avoid division by zero
epsilon = 1e-10
B01[B01 == 0] = epsilon

# Calculate Turbidity (NTU)
Turb = 8.93 * (B03 / B01) - 6.39

# Save function
def save_tif(filename, array, profile):
    profile.update(dtype='float32', count=1)
    with rasterio.open(filename, 'w', **profile) as dst:
        dst.write(array.astype('float32'), 1)

# Save Turbidity result
save_tif('Turb.tif', Turb, profile)

print("Turbidity band saved successfully.")


Turbidity band saved successfully.
